# Expected SARSA and $n$-step bootstrapping 

In this notebook, you will implement expected SARSA and $n$-step bookstrapping described in Chapters 6 and 7 of [Sutton and Barto's book, Introduction to Reinforcement Learning](http://incompleteideas.net/book/the-book-2nd.html).  

### Install dependencies

In [3]:
! pip install numpy pandas

### Imports

In [4]:
import numpy as np
import random
import sys          # We use sys to get the max value of a float
import pandas as pd # We only use pandas for displaying tables nicely
pd.options.display.float_format = '{:,.3f}'.format

### ```World``` class and globals

The ```World``` is a grid represented as a two-dimensional array of characters where each character can represent free space, an obstacle, or a terminal. Each non-obstacle cell is associated with a reward that an agent gets for moving to that cell (can be 0). The size of the world is _width_ $\times$ _height_ characters.

A _state_ is a tuple $(x,y)$.

An empty world is created in the ```__init__``` method. Obstacles, rewards and terminals can then be added with ```add_obstacle``` and ```add_reward```.

To calculate the next state of an agent (that is, an agent is in some state $s = (x,y)$ and performs and action, $a$), ```get_next_state()```should be called.

In [5]:
# Globals:
ACTIONS = ("up", "down", "left", "right") 

# Rewards, terminals and obstacles are characters:
REWARDS = {" ": 0, ".": 0.1, "+": 10, "-": -10}
TERMINALS = ("+", "-") # Note a terminal should also have a reward assigned
OBSTACLES = ("#")

# Discount factor
gamma = 1

# The probability of a random move:
rand_move_probability = 0

class World:  
  def __init__(self, width, height):
    self.width = width
    self.height = height
    # Create an empty world where the agent can move to all cells
    self.grid = np.full((width, height), ' ', dtype='U1')
  
  def add_obstacle(self, start_x, start_y, end_x=None, end_y=None):
    """
    Create an obstacle in either a single cell or rectangle.
    """
    if end_x == None: end_x = start_x
    if end_y == None: end_y = start_y
    
    self.grid[start_x:end_x + 1, start_y:end_y + 1] = OBSTACLES[0]

  def add_reward(self, x, y, reward):
    assert reward in REWARDS, f"{reward} not in {REWARDS}"
    self.grid[x, y] = reward

  def add_terminal(self, x, y, terminal):
    assert terminal in TERMINALS, f"{terminal} not in {TERMINALS}"
    self.grid[x, y] = terminal

  def is_obstacle(self, x, y):
    if x < 0 or x >= self.width or y < 0 or y >= self.height:
      return True
    else:
      return self.grid[x ,y] in OBSTACLES 

  def is_terminal(self, x, y):
    return self.grid[x ,y] in TERMINALS

  def get_reward(self, x, y):
    """ 
    Return the reward associated with a given location
    """ 
    return REWARDS[self.grid[x, y]]

  def get_next_state(self, current_state, action):
    """
    Get the next state given a current state and an action. The outcome can be
    stochastic  where rand_move_probability determines the probability of 
    ignoring the action and performing a random move.
    """    
    assert action in ACTIONS, f"Unknown acion {action} must be one of {ACTIONS}"
    
    x, y = current_state 
    
    # If our current state is a terminal, there is no next state
    if self.grid[x, y] in TERMINALS:
      return None

    # Check of a random action should be performed:
    if np.random.rand() < rand_move_probability:
      action = np.random.choice(ACTIONS)

    if action == "up":      y -= 1
    elif action == "down":  y += 1
    elif action == "left":  x -= 1
    elif action == "right": x += 1

    # If the next state is an obstacle, stay in the current state
    return (x, y) if not self.is_obstacle(x, y) else current_state


## A simple world

We will create a simple $8\times8$ world, where the agent should start in the top-left corner at $(0,0)$ and find its way to the bottom-right corner at $(7,7)$.

In [6]:
world = World(8, 8) # 8 8
world.add_terminal(7, 7, "+") # 7 7

print(world.grid.T)

[[' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' '+']]


## Exercise: Expected SARSA

Implement and test expected SARSA, see page 133 of [Introduction to Reinforcement Learning](http://incompleteideas.net/book/the-book-2nd.html). The implementation of expected SARSA is very similar to regular SARSA, so you can use your SARSA implementation as a starting point for your expected SARSA implementation.

In [7]:
# TODO: Implement your code here -- you need a Q-table to keep track of action 
#       value estimates and a policy-function that returns an epsilon greedy 
#       policy based on your estimates.

epsilon = 0.1
Q = np.random.rand(world.width, world.height, len(ACTIONS))
gamma = 0.9
alpha = 0.05


def SARSA_policy(x,y):
    actions = { a : epsilon/len(ACTIONS) for a in ACTIONS }
    actions[ACTIONS[np.argmax(Q[x,y,:])]] = 1 - epsilon + epsilon/len(ACTIONS)
    return actions

def expected_sarsa(world, policy, start_state):
    steps = 0
    current_state = start_state

    possible_actions = policy(*current_state)
    current_action = random.choices(population=list(possible_actions.keys()), weights=possible_actions.values(), k=1)
    # print(current_action)


    while not world.is_terminal(*current_state):
        steps += 1
        # print(steps)
        # Get the next state from the world
        next_state = world.get_next_state(current_state, current_action[0])

        possible_actions = policy(*next_state)
        
        # Get the reward for performing the action
        reward = world.get_reward(*next_state)
        
        next_action = random.choices(population=list(possible_actions.keys()), weights=possible_actions.values(), k=1)


        x, y = current_state
        nx, ny = next_state

        expected_value = 0
        for action in ACTIONS:
            action_prob = possible_actions[action]
            action_index = ACTIONS.index(action)
            expected_value += action_prob + Q[nx, ny, action_index]

        Q[x,y, ACTIONS.index(current_action[0])] = Q[x,y, ACTIONS.index(current_action[0])] + alpha * (reward + gamma * Q[nx,ny, ACTIONS.index(next_action[0])] - Q[x,y, ACTIONS.index(current_action[0])])
        current_state = next_state
        current_action = next_action


EPISODES = 1000

for episode in range(EPISODES):
    # print(f"Episode {episode + 1 }/{EPISODES}:")
    expected_sarsa(world, SARSA_policy, (0, 0))
    
highest_action = np.full((world.width, world.height), '      ')
for x in range(world.width):
    for y in range(world.height):
        highest_action[x, y] = ACTIONS[np.argmax(Q[x, y])]
highest_action[7, 7] = 'GOAL'
print(pd.DataFrame(highest_action.T))





       0      1      2      3      4      5      6      7
0   down   down   left     up   down   left     up   down
1   down   down  right   left   left   left   left   down
2   down   down   down   left   left   left     up     up
3   down  right   down     up     up     up   down     up
4   down   down   left   left   left     up   down  right
5  right   down     up   down   down   left     up   left
6     up  right   down   down   left   down   down     up
7  right  right  right  right  right  right  right   GOAL


Try your expected SARSA implementation on the $8\times8$ world defined above.

In [8]:
### TODO: try your expected SARSA implementation 

### Test and compare your expected SARSA implementation

Test your implementation on the simple $8 \times 8$ world defined above with, ```rand_move_probability = 0.0```, $\epsilon = 0.1$, and $\gamma = 0.9$. Compare the performance of expected SARSA with normal SARSA. How long does it take for the respective methods to solve the tasks optimally (that is, number of steps that an agent takes from start to finish = _width_ + _height_ $-2 = 14$). 

Remember that for expected SARSA, you can use a learning rate of 1 ($\alpha = 1)$, whereas for SARSA, you should try different learning rates.


In [9]:
rand_move_probability = 0.0
epsilon = 0.1
gamma = 0.9

### TODO: Compare the performance of SARSA (different alphas) and 
###       expected SARSA (alpha = 1). You need to run multiple 
###       experiments (e.g. 100 per setting) and then take the average 
###       to get a proper estimate of the general performance.


## Exercise: $n$-step on-policy SARSA or $n$-step on-policy expected SARSA

Here, you should implement on-policy $n$-step bootstrapping. You can either implement $n$-step SARSA or $n$-step expected SARSA. See Chapter 7 in [Introduction to Reinforcement Learning](http://incompleteideas.net/book/the-book-2nd.html) and page 147 for $n$-step SARSA.

Test you implementation with different values of $n$ either on the $8\times8$ world above or on the ```WindyGridWorld``` from previous lectures. It is up to you to decide how to measure performance: it could, for instance, be the average episode length after a limited number of episodes (for instance $10$), how long it takes to solve the task optimally, like in the exercise on expected SARSA above, or something else.

For the world that you choose, you have to answer the question: "What is a good value for $n$?"

In [51]:
### TODO: Implment and test n-step SARSA or n-step expected SARSA.

epsilon = 0.1
Q = np.random.rand(world.width, world.height, len(ACTIONS))
gamma = 0.9
alpha = 0.05

def e_greedy(x,y):
    actions = { a : epsilon/len(ACTIONS) for a in ACTIONS }
    actions[ACTIONS[np.argmax(Q[x,y,:])]] = 1 - epsilon + epsilon/len(ACTIONS)
    return actions

def nsarsa(world, n, start_state):
    state_buffer = [None] * (n + 1) ## Buffer til at gemme state, action og rewards
    action_buffer = [None] * (n + 1)
    reward_buffer = [None] * (n + 1)

    current_state = start_state
    possible_actions = e_greedy(*current_state)
    current_action = random.choices(
        population=list(possible_actions.keys()),
        weights=possible_actions.values(),
        k=1
    )
    current_action_no_list = current_action[0]

    T = np.inf
    t = 0

    while not world.is_terminal(*current_state) or t < T - 1:

        if t < T:
            # Get the next state from the world
            next_state = world.get_next_state(current_state, current_action_no_list)

            # Get the reward for performing the action
            reward = world.get_reward(*next_state)

            # Gem data i buffer
            state_buffer[t % (n + 1)] = current_state # modulo laver en cirkulær datastruktur så gammel data overskrives
            action_buffer[t % (n + 1)] = current_action
            reward_buffer[t % (n + 1)] = reward
            
            if world.is_terminal(*next_state):
                T = t + 1
            else:
                next_action = e_greedy(*next_state)

                current_state = next_state
                current_action = next_action

        tau = t - n + 1

        if tau >= 0:
            G = np.sum(gamma**(i - tau -1) * reward_buffer[i % (n+1)] for i in range(tau, min(tau + n, T)))

            if tau + n < T:
                G += gamma**n * Q[state_buffer[(tau + n) % (n +1)], action_buffer[(tau + n) % (n + 1)]]
                    
            state_tau = state_buffer[tau % (n + 1)]
            state_tau = tuple(state_tau)
            action_tau = ACTIONS.index(action_buffer[tau % (n + 1)][0])
          
            Q[state_tau, action_tau] += alpha * (G - Q[state_tau, action_tau])

        t += 1



In [52]:
EPISODES = 1000

for episode in range(EPISODES):
    # print(f"Episode {episode + 1 }/{EPISODES}:")
    nsarsa(world, 5, (0, 0))
    
highest_action = np.full((world.width, world.height), '      ')
for x in range(world.width):
    for y in range(world.height):
        highest_action[x, y] = ACTIONS[np.argmax(Q[x, y])]
highest_action[7, 7] = 'GOAL'
print(pd.DataFrame(highest_action.T))




/tmp/ipykernel_8847/310318026.py:55: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  G = np.sum(gamma**(i - tau -1) * reward_buffer[i % (n+1)] for i in range(tau, min(tau + n, T)))


ValueError: operands could not be broadcast together with shapes (1,1,8,8,4) (2,4) 